In [1]:
import torch

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(56180, 500)
        self.fc2 = torch.nn.Linear(500, 50)
        self.fc3 = torch.nn.Linear(50, 21)